In [16]:
# .env의 내용을 읽어서 환경변수로 등록처리
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
# 환경변수 불러오기
import os

NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')

In [19]:
# 네이버 데이터랩 자동차 트렌드 분석 - PC 남자 버전_확정_1
import requests  # (리퀘스츠)
import json      # (제이슨)
import csv       # (씨에스브이)

# 1. 네이버 인증 정보 (실제 값으로 교체해서 사용)
client_id = NAVER_CLIENT_ID       # 네이버 개발자 센터에서 발급
client_secret = NAVER_CLIENT_SECRET

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

# 2. 요청 바디 설정
body = {
    "startDate": "2025-01-01",
    "endDate": "2025-10-31",
    "timeUnit": "month",   # date=일간, week=주간, month=월간
    "keywordGroups": [
        {
            "groupName": "쏘렌토",
            "keywords": ["쏘렌토"]
        },
        {
            "groupName": "카니발",
            "keywords": ["카니발"]
        },
        {
            "groupName": "스포티지",
            "keywords": ["스포티지"]
        },
        {
            "groupName": "셀토스",
            "keywords": ["셀토스"]
        },
        {
            "groupName": "K5",
            "keywords": ["K5"]
        }
    ],
    "device": "pc",   # PC만
    "gender": "m"     # 남자만
    # ages 필터는 생략 → 전체 연령
}

# 3. API 호출
response = requests.post(url, headers=headers, json=body)
response.raise_for_status()  # 에러 응답이면 예외 발생

data = response.json()
print("API 응답 구조 확인:", data.keys())

# 4. 결과를 파이썬 리스트(레코드들)로 정리
records = []  # CSV로 저장할 행 데이터들

for result in data["results"]:
    product_name = result["title"]  # "현대차" 또는 "기아차"
    for row in result["data"]:
        period = row["period"]    # 날짜 (예: 2025-01-01)
        query = row["ratio"]      # 검색 지수(비율값)

        records.append({
            "date": period,
            "product": product_name,  # 현대차 / 기아차
            "device": "pc",
            "gender": "male",     # 사람이 이해하기 쉽게 텍스트로
            "query": query
        })

# 5. CSV 파일로 저장하기
csv_filename = "naver_datalab_data_kia_PC_M1.csv"

with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "product", "device", "gender", "query"])
    writer.writeheader()
    writer.writerows(records)

print(f"CSV 저장 완료: {csv_filename}")



API 응답 구조 확인: dict_keys(['startDate', 'endDate', 'timeUnit', 'results'])
CSV 저장 완료: naver_datalab_data_kia_PC_M1.csv


In [20]:
# 네이버 데이터랩 자동차 트렌드 분석 - PC 남자 버전_확정_1
import requests  # (리퀘스츠)
import json      # (제이슨)
import csv       # (씨에스브이)

# 1. 네이버 인증 정보 (실제 값으로 교체해서 사용)
client_id = NAVER_CLIENT_ID       # 네이버 개발자 센터에서 발급
client_secret = NAVER_CLIENT_SECRET

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

# 0) 차량명 입력
car_name = input("차량명을 입력하세요: ")
car_models = [car_name]

# 1) keywordGroups 생성
keyword_groups = [{"groupName": car_name, "keywords": [car_name]}]

# 2) 성별/디바이스 조합 만들기
genders = {
    "male": "m",
    "female": "f"
}

devices = {
    "pc": "pc",
    "mobile": "mo"
}

# 3) 결과 저장 리스트
records = []

# 4) 모든 성별 × 디바이스 조합 반복
for gender_label, gender_code in genders.items():
    for device_label, device_code in devices.items():

        print(f"=== API 조회 → gender={gender_label}, device={device_label} ===")

        body = {
            "startDate": "2023-01-01",
            "endDate": "2025-10-31",
            "timeUnit": "month",
            "keywordGroups": keyword_groups,
            "device": device_code,
            "gender": gender_code
        }

        # API 호출
        response = requests.post(url, headers=headers, json=body)
        response.raise_for_status()
        data = response.json()

        # 결과 정리
        for result in data["results"]:
            product = result["title"]
            for row in result["data"]:
                records.append({
                    "date": row["period"],
                    "product": product,
                    "device": device_label,   # pc / mobile
                    "gender": gender_label,   # male / female
                    "query": row["ratio"]
                })

# 5) CSV 저장
csv_filename = f"naver_datalab_{car_name}_all.csv"

with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["date", "product", "device", "gender", "query"]
    )
    writer.writeheader()
    writer.writerows(records)

print(f"\nCSV 저장 완료 → {csv_filename}")
print("총 레코드 수:", len(records))

=== API 조회 → gender=male, device=pc ===
=== API 조회 → gender=male, device=mobile ===
=== API 조회 → gender=female, device=pc ===
=== API 조회 → gender=female, device=mobile ===

CSV 저장 완료 → naver_datalab_쏘렌토_all.csv
총 레코드 수: 136


In [ ]:
# 드롭다운 기반 네이버 데이터랩 조회 시스템
import requests
import json
import csv
from ipywidgets import interact, Dropdown

# ================================
# 1. 차량 리스트 40개 준비
# ================================
car_list = [
    "아반떼", "그랜저", "싼테페", "쏘나타 디 엣지", "투싼",
    "디 올 뉴 팰리세이드", "포터2", "코나", "스타리아", "아이오닉 5",
    "캐스퍼 일렉트릭", "아이오닉 9", "디 올 뉴 넥쏘", "포터2 일렉트릭", "베뉴",
    "캐스퍼", "코나 일렉트릭", "더 뉴 아이오닉6", "ST1", "아반떼 N",
    "아이오닉 5 N", "쏘렌토", "카니발", "스포티지", "셀토스",
    "K5", "봉고 3", "레이", "K8", "모닝",
    "PV5", "EV3", "EV5", "니로", "레이 EV",
    "EV6", "EV4", "타스만", "EV9", "K9", "니로 EV"
]

# 드롭다운 구성
dropdown = Dropdown(
    options=car_list,
    description='차량 선택:',
    disabled=False,
)

# ================================
# 2. 네이버 인증 정보
# ================================
client_id = NAVER_CLIENT_ID
client_secret = NAVER_CLIENT_SECRET
url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

# ================================
# 3. 트렌드 조회 함수
# ================================
def fetch_trend(car_name):

    keyword_groups = [{"groupName": car_name, "keywords": [car_name]}]

    genders = {"male": "m", "female": "f"}
    devices = {"pc": "pc", "mobile": "mo"}

    records = []

    for gender_label, gender_code in genders.items():
        for device_label, device_code in devices.items():

            body = {
                "startDate": "2023-01-01",
                "endDate": "2025-10-31",
                "timeUnit": "month",
                "keywordGroups": keyword_groups,
                "device": device_code,
                "gender": gender_code
            }

            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            data = response.json()

            for result in data["results"]:
                for row in result["data"]:
                    records.append({
                        "date": row["period"],
                        "product": car_name,
                        "device": device_label,
                        "gender": gender_label,
                        "query": row["ratio"]
                    })

    # CSV 저장
    csv_filename = f"naver_datalab_{car_name}_all.csv"
    with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["date", "product", "device", "gender", "query"])
        writer.writeheader()
        writer.writerows(records)

    print(f"CSV 저장 완료 → {csv_filename}")
    print(f"총 데이터 건수: {len(records)}")


# ================================
# 4. 드롭다운과 함수 연결
# ================================
interact(fetch_trend, car_name=dropdown)


In [ ]:
# app.py - Streamlit 기반 네이버 데이터랩 자동차 트렌드 분석
import requests
import json
import csv
import pandas as pd
import streamlit as st

# ================================
# 1. 차량 리스트 40개 준비
# ================================
car_list = [
    "아반떼", "그랜저", "싼테페", "쏘나타 디 엣지", "투싼",
    "디 올 뉴 팰리세이드", "포터2", "코나", "스타리아", "아이오닉 5",
    "캐스퍼 일렉트릭", "아이오닉 9", "디 올 뉴 넥쏘", "포터2 일렉트릭", "베뉴",
    "캐스퍼", "코나 일렉트릭", "더 뉴 아이오닉6", "ST1", "아반떼 N",
    "아이오닉 5 N", "쏘렌토", "카니발", "스포티지", "셀토스",
    "K5", "봉고 3", "레이", "K8", "모닝",
    "PV5", "EV3", "EV5", "니로", "레이 EV",
    "EV6", "EV4", "타스만", "EV9", "K9", "니로 EV"
]

# ================================
# 2. 네이버 인증 정보
# ================================
NAVER_CLIENT_ID = "YOUR_NAVER_CLIENT_ID"
NAVER_CLIENT_SECRET = "YOUR_NAVER_CLIENT_SECRET"

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    "Content-Type": "application/json"
}

# ================================
# 3. 트렌드 조회 함수
# ================================
def fetch_trend(car_name):

    keyword_groups = [{"groupName": car_name, "keywords": [car_name]}]

    genders = {"male": "m", "female": "f"}
    devices = {"pc": "pc", "mobile": "mo"}

    records = []

    for gender_label, gender_code in genders.items():
        for device_label, device_code in devices.items():

            body = {
                "startDate": "2023-01-01",
                "endDate": "2025-10-31",
                "timeUnit": "month",
                "keywordGroups": keyword_groups,
                "device": device_code,
                "gender": gender_code
            }

            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            data = response.json()

            for result in data["results"]:
                for row in result["data"]:
                    records.append({
                        "date": row["period"],
                        "product": car_name,
                        "device": device_label,
                        "gender": gender_label,
                        "query": row["ratio"]
                    })

    return records


# ================================
# 4. Streamlit UI
# ================================
def main():

    st.title("🚗 네이버 데이터랩 자동차 검색 트렌드 분석")
    st.write("차량을 선택하면 성별·디바이스별 검색 지수 데이터를 가져옵니다.")

    # 차량 선택 selectbox
    car_name = st.selectbox("차량을 선택하세요", car_list)

    # ▼ 버튼 클릭 시 조회 실행
    if st.button("트렌드 조회"):

        with st.spinner("데이터 조회 중..."):

            try:
                records = fetch_trend(car_name)
            except Exception as e:
                st.error(f"데이터 수집 실패: {e}")
                return

        # DataFrame 변환
        df = pd.DataFrame(records)

        st.success("데이터 조회 완료!")
        st.write(f"총 데이터 건수: **{len(df)}**")

        # 데이터프레임 화면 표시
        st.dataframe(df)

        # CSV 다운로드 버튼
        csv_filename = f"naver_datalab_{car_name}_all.csv"
        csv_data = df.to_csv(index=False, encoding="utf-8-sig")

        st.download_button(
            label="📥 CSV 다운로드",
            data=csv_data,
            file_name=csv_filename,
            mime="text/csv"
        )


if __name__ == "__main__":
    main()


In [ ]:
# 네이버 데이터랩 자동차 트렌드 분석 - 차량명 입력 + 브랜드 컬럼 버전
import requests  # (리퀘스츠)
import json      # (제이슨)
import csv       # (씨에스브이)

# ================================
# 1. 브랜드별 차량 리스트 (브랜드 판별용)
# ================================
HYUNDAI_MODELS = [
    "아반떼", "그랜저", "싼타페", "쏘나타 디 엣지", "투싼",
    "디 올 뉴 팰리세이드", "포터2", "코나", "스타리아", "아이오닉 5",
    "캐스퍼 일렉트릭", "아이오닉 9", "디 올 뉴 넥쏘", "포터2 일렉트릭", "베뉴",
    "캐스퍼", "코나 일렉트릭", "더 뉴 아이오닉6", "ST1", "아반떼 N",
    "아이오닉 5 N"
]

KIA_MODELS = [
    "쏘렌토", "카니발", "스포티지", "셀토스",
    "K5", "봉고 3", "레이", "K8", "모닝",
    "PV5", "EV3", "EV5", "니로", "레이 EV",
    "EV6", "EV4", "타스만", "EV9", "K9", "니로 EV"
]

# (선택) 전체 리스트 – 필요하면 쓸 수 있음
car_list = HYUNDAI_MODELS + KIA_MODELS

def get_brand(car_name: str) -> str:
    """차량명으로 HYUNDAI / KIA 브랜드 구분하는 함수"""
    if car_name in HYUNDAI_MODELS:
        return "HYUNDAI"
    elif car_name in KIA_MODELS:
        return "KIA"
    else:
        return "UNKNOWN"  # 리스트에 없을 때 대비


# ================================
# 2. 네이버 인증 정보
# ================================
client_id = NAVER_CLIENT_ID
client_secret = NAVER_CLIENT_SECRET
url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
    "Content-Type": "application/json"
}

# ================================
# 3. 트렌드 조회 함수
# ================================
def fetch_trend(car_name):

    keyword_groups = [{"groupName": car_name, "keywords": [car_name]}]

    genders = {"male": "m", "female": "f"}
    devices = {"pc": "pc", "mobile": "mo"}

    records = []

    # 차량명으로 브랜드 결정 (HYUNDAI / KIA)
    brand = get_brand(car_name)

    for gender_label, gender_code in genders.items():
        for device_label, device_code in devices.items():

            body = {
                "startDate": "2023-01-01",
                "endDate": "2025-10-31",
                "timeUnit": "month",
                "keywordGroups": keyword_groups,
                "device": device_code,
                "gender": gender_code
            }

            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            data = response.json()

            for result in data["results"]:
                for row in result["data"]:
                    records.append({
                        "date": row["period"],     # 1열
                        "brand": brand,            # 2열 (새로 추가)
                        "product": car_name,       # 3열
                        "device": device_label,    # 4열
                        "gender": gender_label,    # 5열
                        "query": row["ratio"]      # 6열
                    })

    # CSV 저장
    csv_filename = f"naver_datalab_{car_name}_all.csv"
    with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["date", "brand", "product", "device", "gender", "query"]
        )
        writer.writeheader()
        writer.writerows(records)

    print(f"CSV 저장 완료 → {csv_filename}")
    print(f"총 데이터 건수: {len(records)}")


# ================================
# 4. 차량명 입력 받아서 실행
# ================================
if __name__ == "__main__":
    car_name = input("차량명을 입력하세요: ")

    # (선택) 유효성 체크
    # if car_name not in car_list:
    #     print("⚠️ 등록된 리스트에 없는 차량입니다. 철자/띄어쓰기를 확인해 주세요.")
    # else:
    #     fetch_trend(car_name)

    # 간단하게 바로 실행
    fetch_trend(car_name)


CSV 저장 완료 → naver_datalab_아반테_all.csv
총 데이터 건수: 136
